# Find the sets of 5 words with no char in common

In [ ]:
import os
import sys
import requests
import numpy as np
from tqdm.notebook import tqdm
from collections import defaultdict, Counter
from string import ascii_lowercase
from typing import List, Set, Dict
char_map = {c:i for i,c in enumerate(ascii_lowercase)}

## Get word list

In [ ]:
def get_words():
    # Source: https://gist.github.com/subhrm/5362767af06597bd1e216c59b760f6cb
    url="https://gist.githubusercontent.com/subhrm/5362767af06597bd1e216c59b760f6cb/raw/6bfa15d263d6d5b63840a8e5b64e04b382fdb079/valid-wordle-words.txt"

    # Download word list
    resp = requests.get(url)
    print(f"{resp.status_code=}")
    resp.raise_for_status()

    # process the downloaded list
    word_list = list(set([w for w in resp.text.split("\n") if len(w)==5]))
    print(f"{len(word_list)=}")
    return word_list

word_list = get_words()

In [ ]:
# keep only words with unique chars
word_list = [w for w in word_list if len(set(w)) == 5]
print(f"{len(word_list)=}")

## Group anagrams

In [ ]:
def word_to_num(word):
    return sum( (1<<char_map[c]) for c in word)

# test
print(f"{word_to_num("abc")=}")
print(f"{word_to_num("ac")=}")
assert word_to_num("abc") == word_to_num("bac")


## Construct inveretd index for anagrams

In [ ]:
anagrams = defaultdict(list)
for w in word_list:
    num = word_to_num(w)
    anagrams[num].append(w)

unique_list = sorted(anagrams.keys())
print(len(unique_list))

In [ ]:
# construct inverted negative index of anagrams
neg_one_map = defaultdict(list)
for i in range(25):
    for j in range(i+1, 26):
        key = (1<<i) | (1<<j)
        neg_one_map[key] = [ w for w in unique_list if w&key == 0]

neg_one_map_keys = sorted(neg_one_map.keys(), key=lambda x: len(neg_one_map[x]))
print(f"{len(neg_one_map_keys)=}")

In [ ]:
print([len(neg_one_map[k]) for k in neg_one_map_keys[:10]])
print([len(neg_one_map[k]) for k in neg_one_map_keys[-10:]])

## Counstruct 2 set

In [ ]:
two_set = defaultdict(list)

for a in tqdm(unique_list):
    for key in neg_one_map_keys:
        if (a & key) == key:
            for b in neg_one_map[key]:
                if b>a and (a & b) == 0:
                    key = a|b
                    two_set[key].append((a,b))
            break
two_set_keys = sorted(two_set.keys())

In [ ]:
print(f"{len(two_set_keys)=:,}")

In [ ]:
samples = 2
for k in two_set_keys[:samples]:
    v = two_set[k]
    print(f"{k=} {len(v)=}")
    for a,b in v[:5]:
        print(f"{anagrams[a]} x {anagrams[b]}")
    print("="*40)

## Construct 3-set

In [ ]:
three_set = defaultdict(list)

for a in tqdm(two_set_keys):
    for key in neg_one_map_keys:
        if (a & key) == key:
            for b in neg_one_map[key]:
                if (a & b) == 0:
                    key = a|b
                    three_set[key].append((a,b))
            break

print(f"{len(three_set)=:,}")

In [ ]:
three_set_keys = sorted(three_set.keys())
print(f"{len(three_set_keys)=:,}")

## Compute 5 set

In [ ]:
from itertools import combinations
from functools import reduce

In [ ]:
five_set = defaultdict(list)
super_key = (1<<26) - 1
with tqdm(total=len(three_set_keys)) as pbar:
    step = 500
    for idx,a in enumerate(three_set_keys):
        mask = super_key ^ a
        pos = [x for x in range(26) if (mask & (1<<x)) > 0]
        # assert len(pos) == 11
        comb = combinations(pos, 10)
        # assert len(comb) == 11
        for p in comb:
            req_key = reduce(lambda x,y: x|(1<<y), p, 0)
            if req_key > a and req_key in two_set:
                key = a | req_key
                five_set[key].append((a,req_key))

        if idx%step == 0:
            pbar.update(step)

print(f"{len(five_set)=:,}")

## Display Results

In [ ]:
def expand_two_key(key):
    res = set()
    for a,b in two_set[key]:
        res.add(tuple(sorted((a,b))))

    return list(res)

def expand_three_key(key):
    res = set()
    for two_key,c in three_set[key]:
        for a,b in expand_two_key(two_key):
            res.add(tuple(sorted((a,b,c))))

    return list(res)

def print_five_set(k):
    res = set()
    for three_key , two_key in five_set[k]:
        threes_list = expand_three_key(three_key)
        twos_list = expand_two_key(two_key)

        for threes in threes_list:
            for twos in twos_list:
                res.add(tuple(sorted(threes + twos)))

    final_list = []
    for t in sorted(res):
        final_list.append(" | ".join(",".join(anagrams[k]) for k in t))
    return final_list

In [ ]:
for k in five_set.keys():
    print("="*40)
    results = print_five_set(k)
    print(f"{k=} {len(results)=}")
    for r in results:
        print(f"{r}")
